In [1]:
import pymongo
import time
import pandas as pd
import json
import nltk

# Conexão com o MongoDB

In [2]:
client = pymongo.MongoClient()

# database
db = client['IWD2021']  

# collection
collection = db.tweets_es

In [3]:
pipeline = [
    {
        '$project': {
            'tweet_id': '$id_str', 
            'date': {
                '$dateFromString': {
                    'dateString': '$created_at'
                }
            }, 
            'tweet_text': {
                '$ifNull': [
                    '$retweeted_status.full_text', '$full_text'
                ]
            }, 
            'num_likes': '$favorite_count', 
            'num_retweets': '$retweet_count', 
            'quoted_status_id': {
                '$ifNull': [
                    '$quoted_status_id_str', None
                ]
            }, 
            'retweeted_status_id': {
                '$ifNull': [
                    '$retweeted_status.id_str', None
                ]
            }, 
            'reply_to_user': {
                '$ifNull': [
                    '$in_reply_to_user_id_str', None
                ]
            }, 
            'reply_to_status': {
                '$ifNull': [
                    '$in_reply_to_status_id_str', None
                ]
            }, 
            'user_id': '$user.id_str', 
            'screen_name': '$user.screen_name', 
            'followers': '$user.followers_count', 
            'following': '$user.friends_count'
        }
    }, {
        '$project': {
            '_id': 0, 
            'tweet_id': 1, 
            'date': {
                '$dateToString': {
                    'format': '%d/%m/%Y %H:%M:%S', 
                    'date': '$date'
                }
            }, 
            'tweet_text': 1, 
            'num_likes': 1, 
            'num_retweets': 1, 
            'quoted_status_id': 1, 
            'retweeted_status_id': 1, 
            'reply_to_user': 1, 
            'reply_to_status': 1, 
            'user_id': 1, 
            'screen_name': 1, 
            'followers': 1, 
            'following': 1
        }
    }
]

In [4]:
time_start = time.time()
document = list(collection.aggregate(pipeline=pipeline))
time_end = time.time()

print(
    f'Tempo de execução: {time.strftime("%H:%M:%S", time.gmtime(time_end - time_start))}')
print(f'{len(document)} documentos selecionados')

Tempo de execução: 00:00:37
1441175 documentos selecionados


In [5]:
print(f'Amostra de um documento:\n')
print(json.dumps(document[0], sort_keys=False, indent=3, ensure_ascii=False))

Amostra de um documento:

{
   "tweet_id": "1368641547479248902",
   "tweet_text": "Time for a Female UN Secretary-General? https://t.co/xH2moPGhHd #IWD2021 https://t.co/0NJjuCeReH",
   "num_likes": 0,
   "num_retweets": 53,
   "quoted_status_id": null,
   "retweeted_status_id": "1368634417804107776",
   "reply_to_user": null,
   "reply_to_status": null,
   "user_id": "2568577971",
   "screen_name": "AMalegrooming",
   "followers": 757,
   "following": 1234,
   "date": "07/03/2021 19:15:47"
}


In [6]:
df = pd.DataFrame(document)
df = df[['tweet_id', 'date', 'tweet_text', 'num_likes', 'num_retweets',
                   'retweeted_status_id', 'quoted_status_id', 'reply_to_user', 'reply_to_status',
                   'user_id', 'screen_name', 'followers', 'following']]
df.head()

,tweet_id,date,tweet_text,num_likes,num_retweets,retweeted_status_id,quoted_status_id,reply_to_user,reply_to_status,user_id,screen_name,followers,following
0,1368641547479248902,07/03/2021 19:15:47,Time for a Female UN Secretary-General? https:...,0,53,1368634417804107776,None,None,None,2568577971,AMalegrooming,757,1234
1,1368641811749732360,07/03/2021 19:16:50,Así de simple 👇 https://t.co/a8VGX5MjQP,0,0,None,1368525594737131520,None,None,273633856,Naim_Lopez,291,822
2,1368642439657963520,07/03/2021 19:19:20,📣Trabajemos para romper las barreras sistémic...,0,133,1368622576394731525,None,None,None,4417466061,zanaines,4557,4255
3,1368642748023246851,07/03/2021 19:20:33,Time for a Female UN Secretary-General? https:...,0,53,1368634417804107776,None,None,None,705760236779474944,drpicka,100,482
4,1368642774942314505,07/03/2021 19:20:40,#EFEFotos | DÍA DE LA MUJER\n\nComo si no bast...,0,7,1368585986788253697,None,None,None,1066770045357998080,latarjetamorada,199,668


In [7]:
print(f'Amostra de um tweet antes do pré-processamento:\n\n{df.tweet_text[0]}')

Amostra de um tweet antes do pré-processamento:

Time for a Female UN Secretary-General? https://t.co/xH2moPGhHd #IWD2021 https://t.co/0NJjuCeReH


In [8]:
df.to_csv('tweets_es.csv')

### Pré-processamento

In [9]:
# remover links
df['clean_text'] = df['tweet_text'].str.replace(r"http\S+","") 

df.clean_text[0]

'Time for a Female UN Secretary-General?  #IWD2021 '

In [10]:
# remover mentions
df['clean_text'] = df['clean_text'].str.replace(r"@\S+","")

df.clean_text[0]

'Time for a Female UN Secretary-General?  #IWD2021 '

In [11]:
# remover hashtags
df['clean_text'] = df['clean_text'].str.replace(r"#\S+"," ")

df.clean_text[0]

'Time for a Female UN Secretary-General?    '

In [12]:
# remover quebra de linhas
df['clean_text'] = df['clean_text'].str.replace(r"\n"," ")

df.clean_text[0]

'Time for a Female UN Secretary-General?    '

In [13]:
# remover pontuações
df['clean_text'] = df['clean_text'].str.replace(r"[^\w\s#]|_"," ")

df.clean_text[0]

'Time for a Female UN Secretary General     '

In [14]:
# remover números
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"\b(?:[0-9]*)\b","")

df.clean_text[0]

'Time for a Female UN Secretary General'

In [15]:
# remover espaços duplos
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"\s{2,}"," ")

df.clean_text[0]

'Time for a Female UN Secretary General'

### converter todas as letras para minúsculas

In [16]:
df['clean_text'] = df['clean_text'].str.lower()

df.clean_text[0]

'time for a female un secretary general'

### remover letras que se repetem em sequência mais de 3x

In [17]:
import string
alphabet = list(string.ascii_lowercase)

for letter in alphabet:
    pattern = '{}{}'.format(letter, '{3,}')
    df['clean_text'] = df['clean_text'].str.replace(r'{}'.format(pattern), letter)
    
df.clean_text[0]

'time for a female un secretary general'

### remover stpwords

In [18]:
with open('../stopwords/spanish.txt', 'r') as file:
    stopwords = file.readlines()
    stopwords = [sw.replace('\n','') for sw in stopwords]

def remover_stopwords(texto):
    palavras = [i for i in texto.split() if not i in stopwords]
    return (" ".join(palavras))

df['clean_text'] = df['clean_text'].apply(remover_stopwords)

df.clean_text[0]

'time for female secretary general'

### remover palavras com 2 caracteres ou menos

In [19]:
def remover_len_2(texto):
    palavras = [i for i in texto.split() if len(i) > 2]
    return (" ".join(palavras))

df['clean_text'] = df['clean_text'].apply(remover_len_2)

df.clean_text[0]

'time for female secretary general'

In [20]:
print(f'Amostra de um tweet depois do pré-processamento:\n\n{df.clean_text[0]}')

Amostra de um tweet depois do pré-processamento:

time for female secretary general


In [21]:
def reduzir_radical(texto):
    stemmer = nltk.stem.RSLPStemmer()
    palavras = [stemmer.stem(i) for i in texto.split()]
    return (" ".join(palavras))

In [22]:
reduzir_radical(df.tweet_text[0])

'tim for a femal un secretary-general? https://t.co/xh2mopghhd #iwd2021 https://t.co/0njjucereh'

In [23]:
# df['tweet_text'] = df['tweet_text'].apply(reduzir_radical)

In [24]:
import unicodedata

def remover_acentuacao(text):
    # Unicode normalize transforma um caracter em seu equivalente em latin.
    nfkd = unicodedata.normalize('NFKD', text)
    palavra_sem_acento = u"".join([c for c in nfkd if not unicodedata.combining(c)])
    return (palavra_sem_acento)

In [25]:
remover_acentuacao(df.tweet_text[0])

'Time for a Female UN Secretary-General? https://t.co/xH2moPGhHd #IWD2021 https://t.co/0NJjuCeReH'

In [26]:
df['clean_text'] = df['clean_text'].apply(remover_acentuacao)

In [27]:
# remover non ascii caracteres
df['clean_text'] = df['clean_text'].str.strip().str.replace(r"[^a-zA-z0-9#|\s]","")

df.clean_text[0]

'time for female secretary general'

In [28]:
print(f'Amostra de um tweet depois do pré-processamento:\n\n{df.clean_text[0]}')

Amostra de um tweet depois do pré-processamento:

time for female secretary general


### Etapa opcional

O atributo 'lang' presentes nos documentos podem ter sido atribuidos erroneamente. Objetivando eliminar tweets que possam ser de um idioma não requisitado, desenvolveram-se 2 funções para ajudar a confirmar o idioma destes.

Para desabilitar warnings:

In [29]:
# import logging
# logging.disable(logging.WARNING)

#### 1. Usando polyglot

In [30]:
# import polyglot
# from polyglot.detect import Detector

In [31]:
# def get_polyglot_language(text):
#     try:
#         detector = Detector(text)
#     except polyglot.detect.base.UnknownLanguage:
#         lang = 'Undefined'
#     else:
#         lang = detector.language.name
    
#     return (lang)

In [32]:
# get_polyglot_language(df['tweet_text'][0])

In [33]:
# df['polyglot_lang'] = df['tweet_text'].apply(get_polyglot_language)

In [34]:
# df['polyglot_lang'].value_counts()

#### 2. Usando textblob

In [35]:
# import textblob
# from textblob import TextBlob
# import pycountry

In [36]:
# def get_textblob_language(text):
#     try:
#         time.sleep(0.2)
#         b = TextBlob(text)
#         language = pycountry.languages.get(alpha_2=b.detect_language())
#         language_name = language.name
#     except textblob.exceptions.TranslatorError:
#         # language_name = "Deu ruim"
#         language_name = 'Undefined'
    
#     return(language_name) 

In [37]:
# get_textblob_language(df['tweet_text'][0])

In [38]:
# df['textblob_lang'] = df['tweet_text'].apply(get_textblob_language)

Para reabilitar warnings:

In [39]:
# logging.disable(logging.NOTSET)

In [40]:
# df.shape

# Criação do arquivo .csv

### Cenário 1: base completa

Caso tenha se utilizado umas das duas funções - **get_polyglot_language**, por exemplo - use o seguinte trecho para gerar o arquivo .csv:

In [41]:
# index = df['polyglot_lang'] == 'Portuguese'
# new_df = df[index]
# new_df = new_df.drop('polyglot_lang', axis=1)

In [42]:
# new_df.to_csv('cenario1_8M2020_polyglot_tweets_pt.csv', index=False)

Caso contrário:

In [43]:
# df = df.drop('polyglot_lang', axis=1)

In [44]:
df = df[df['clean_text'] != '']

In [45]:
df.shape

(1437323, 14)

In [46]:
df.to_csv('cenario1_8M2021_tweets_es.csv', index=False)

criação do arquivo .txt (somente com os textos dos tweets):

In [47]:
import numpy as np

In [48]:
tweets_text = np.array(df['clean_text'])

In [49]:
np.savetxt(fname='cenario1_8M2021_tweets_es.txt', X=tweets_text, fmt='%s')

## Cenário 2: tweets sem duplicatas

In [50]:
df.drop_duplicates(['clean_text'], inplace = True)

In [51]:
df.shape

(266214, 14)

In [52]:
df.to_csv('cenario2_8M2021_tweets_es.csv', index=False) 

In [53]:
tweets_text = np.array(df['clean_text'])

In [54]:
np.savetxt(fname='cenario2_8M2021_tweets_es.txt', X=tweets_text, fmt='%s')